# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [18]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
#!pip install python-dotenv

In [19]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [20]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [23]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

#### Call the ChatModel (the LLM)

In [24]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="That's a great choice! Blue is often associated with calmness, serenity, and tranquility. What do you like most about the color blue?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-17438b42-d52f-4b38-b95a-011d0f71a06b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 32, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

In [25]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I cannot determine your favorite color as I do not have information about your personal preferences.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e2cfafd-8407-4add-b076-88fb07ecec4d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 22, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Check if the Chatbot remembers your favorite color.

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain_community

In [26]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [27]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [28]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [29]:
chain = LLMChain(
    llm = chatbot,
    prompt = prompt,
    memory = memory,
)

LLMChain(memory=ConversationBufferMemory(chat_memory=<langchain_community.chat_message_histories.file.FileChatMessageHistory object at 0x10c5a3310>, return_messages=True, memory_key='messages'), verbose=False, prompt=ChatPromptTemplate(input_variables=['content', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], t

In [30]:
chain.invoke("hi ")

{'content': 'hi ',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [ ]:
chain.invoke("hi ")

In [31]:
chain.invoke("my name is prince katiyar  ")

{'content': 'my name is prince katiyar  ',
 'messages': [HumanMessage(content='hi ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': "It's nice to meet you, Prince Katiyar! How can I help you today?"}

In [32]:
chain.invoke("My favorite color is blue")

{'content': 'My favorite color is blue',
 'messages': [HumanMessage(content='hi ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is prince katiyar  ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It's nice to meet you, Prince Katiyar! How can I help you today?", additional_kwargs={}, response_metadata={})],
 'text': "Blue is a great color choice! It's often associated with calmness and stability. Do you have a specific reason why blue is your favorite color?"}

In [33]:
chain.invoke("what is my name")

{'content': 'what is my name',
 'messages': [HumanMessage(content='hi ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is prince katiyar  ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It's nice to meet you, Prince Katiyar! How can I help you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My favorite color is blue', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Blue is a great color choice! It's often associated with calmness and stability. Do you have a specific reason why blue is your favorite color?", additional_kwargs={}, response_metadata={})],
 'text': 'Your name is Prince Katiyar.'}

In [34]:
chain.invoke("what is my faviorite color")

{'content': 'what is my faviorite color',
 'messages': [HumanMessage(content='hi ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is prince katiyar  ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It's nice to meet you, Prince Katiyar! How can I help you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My favorite color is blue', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Blue is a great color choice! It's often associated with calmness and stability. Do you have a specific reason why blue is your favorite color?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Prince Katiyar.', additional_kwargs={}, response_metadata={})],
 'text': 'Your favorite color is blue.'}

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using chatGPT API.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-chatbot.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-chatbot.py